<p style="text-align: center;">Multi-Layer Perceptron (MLP) example in Keras <br> for house sales price prediction<br>using ONLY categorial features
</p>

<p style="text-align: center;">
 by Prof. Dr.-Ing. Jürgen Brauer, http://www.juergenbrauer.org
</p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Retrieve-categorial-feature-columns-from-data" data-toc-modified-id="Retrieve-categorial-feature-columns-from-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Retrieve categorial feature columns from data</a></span></li><li><span><a href="#Show-for-a-categorial-column-possible-values" data-toc-modified-id="Show-for-a-categorial-column-possible-values-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Show for a categorial column possible values</a></span></li><li><span><a href="#One-hot-encoding-for-categorial-columns" data-toc-modified-id="One-hot-encoding-for-categorial-columns-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>One-hot encoding for categorial columns</a></span></li><li><span><a href="#Analyse-datatype-of-one-hot-encoded-columns" data-toc-modified-id="Analyse-datatype-of-one-hot-encoded-columns-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analyse datatype of one-hot encoded columns</a></span></li><li><span><a href="#Encoding-all-categorial-feature-columns-by-one-hot-encoding" data-toc-modified-id="Encoding-all-categorial-feature-columns-by-one-hot-encoding-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Encoding all categorial feature columns by one-hot encoding</a></span></li><li><span><a href="#Scaling-the-data-and-preparing-training-matrices" data-toc-modified-id="Scaling-the-data-and-preparing-training-matrices-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Scaling the data and preparing training matrices</a></span></li><li><span><a href="#Building-the-MLP-model-and-training-it" data-toc-modified-id="Building-the-MLP-model-and-training-it-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Building the MLP model and training it</a></span></li><li><span><a href="#Testing-the-trained-MLP" data-toc-modified-id="Testing-the-trained-MLP-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Testing the trained MLP</a></span></li><li><span><a href="#Predicting-house-prices-for-the-Kaggle-competition" data-toc-modified-id="Predicting-house-prices-for-the-Kaggle-competition-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Predicting house prices for the Kaggle competition</a></span></li></ul></div>

# Retrieve categorial feature columns from data

In [ ]:
import pandas as pd
train_data = pd.read_csv("kaggle_dataset_house_prices/train.csv")
test_data  = pd.read_csv("kaggle_dataset_house_prices/test.csv")

# save the house Ids, since we need them later
# for our Kaggle submission
test_house_ids = test_data.values[:,0] # get the IDs from the original Pandas DataFrame

train_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

gt_saleprice = train_data["SalePrice"]
train_data = train_data.select_dtypes(exclude=['number'])
test_data = test_data.select_dtypes(exclude=['number'])

print(train_data.dtypes)

# Show for a categorial column possible values

It is interesting to see which values a categorial variable can have and how often these individual categorial values occur. Let's choose a categorial variable, e.g. `GarageQual` and get all the values that can appear and how often:

In [ ]:
print( train_data['GarageQual'].value_counts() )

This corresponds to the 6 possible values of the garage quality mentioned in `data_description.txt`:

    Ex	Excellent
    Gd	Good
    TA	Typical/Average
    Fa	Fair
    Po	Poor
    NA	No Garage (which we mapped using fillna() to 0)

In [ ]:
labels = train_data['GarageQual'].astype('category').cat.categories.tolist()
print(labels)
print("labels has type",type(labels))

This allows us to plot a nice pie chart for showing visually how often a certain value of category appears in the dataset:

In [ ]:
import matplotlib.pyplot as plt

labels = train_data['GarageQual'].astype('category').cat.categories.tolist()
counts = train_data['GarageQual'].value_counts()
sizes = [counts[var_cat] for var_cat in labels]

plt.figure()
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%')
ax1.axis('equal')
plt.show()

# One-hot encoding for categorial columns

A simple approach is to use a one-hot encoding to encode categorial features:

In [ ]:
garagequal_only = train_data["GarageQual"]
print(garagequal_only)
garagequal_only_hot = pd.get_dummies(garagequal_only,
                                     columns=['GarageQual'],
                                     prefix = 'GarageQuality')
print(garagequal_only_hot)

# Analyse datatype of one-hot encoded columns

For one `GarageQual` categorial feature column which had data type "object" and where the categorial feature could have six different values [0, 'Ex', 'Fa', 'Gd', 'Po', 'TA'], we made six one-hot encoded feature columns.

In column `Gd` (Good) we will see a `1` for a house, if the garage quality is classified as "good" and otherwise we will see a `0`.

But what is the data type of this new feature columns? Let's see!

In [ ]:
print("garagequal_only_hot is a ", type(garagequal_only_hot))
print(garagequal_only_hot.dtypes)

So the data type is an unsigned integer. This makes sense!

# Encoding all categorial feature columns by one-hot encoding

Now there are ...

In [ ]:
print(len(train_data.dtypes))

... categorial feature columns in our original Pandas Dataframe `train_data`. Fortunately, we can replace all feature columns in one step:

In [ ]:
train_data_hot = pd.get_dummies(train_data)
print(train_data_hot)

This shows how powerful Pandas is! One command does all the work for us. However, we can see that e.g. for the feature `MSZoning` which can have the values

    A   Agriculture
    C   Commercial
    FV  Floating Village Residential
    I   Industrial
    RH  Residential High Density
    RL  Residential Low Density
    RP  Residential Low Density Park 
    RM  Residential Medium Density

according to `data_description.txt` only 5 columns were generated, although there are in principal 8 possible values `(A,C,FV,I,RH,RL,RP,RM)` for this categorial feature:

    MSZoning_C (all) 	MSZoning_FV 	MSZoning_RH 	MSZoning_RL 	MSZoning_RM
   
What's the reason? Well, Pandas can only see the values that are really there in the table! Not more! It does not know, that these feature can in principle also have the value "A", if there is not a single row in the data frame where this value appears.

In [ ]:
labels_mszoning = train_data['MSZoning'].astype('category').cat.categories.tolist()
print(labels_mszoning)

For this we have to make sure, that the training data columns and the test data columns are encoded in the same way! Let's check this!

In [ ]:
train_data_hot = pd.get_dummies(train_data)
test_data_hot = pd.get_dummies(test_data)
print("Shape of train_data_hot is", train_data_hot.shape)
print("Shape of test_data_hot is", test_data_hot.shape)

Ups! We can see that there are in the test table only 256 compared to 268 columns, i.e., there are fewer values of some categorial features and for this, Pandas uses another one-hot encoding for the test data features as for the training data features! This is bad! :( 

Why is this a problem? We cannot use training vectors of size 268 and later give the network 256 dimensional input vectors for inference, where the individual entries of this 256D input vectors also have other meanings compared to the individual entries of the 268D vectors used during training!

What can we do to solve this problem? ... I personally just went to get a coffee ... Then I had the idea! We want `get_dummies` to see ALL possible values that appear in my training and in my test data. So let's build a new dataset, where we stack...

    train.csv (1460 rows)
    test.csv (1459 rows)
    
... both data sets, then call `get_dummies` to do the 1-hot encoding on the fused data frame and then split the 1-hot encoded data frame again into two parts (training and testing). Coffee is the driving force behind innovation! ;-)

In [ ]:
print("Shape of train_data is ", train_data.shape)
print("Shape of test_data is ", test_data.shape)
print("Column names of train_data are:\n", train_data.columns.values)
print("Column names of test_data are:\n", test_data.columns.values)
frames = [train_data, test_data]
fused_df = pd.concat(frames)
#print(fused_df)
print("Shape of fused_df is", fused_df.shape)

# now do the one-hot encoding
fused_df_hot_encoded = pd.get_dummies(fused_df)
print("Shape of fused_df_hot_encoded is", fused_df_hot_encoded.shape)

# now split the data frame into two data frames again
# with 1460 and 1459 rows
train_data_hot_encoded = fused_df_hot_encoded[0:1460]
test_data_hot_encoded  = fused_df_hot_encoded[1460:]
print("Shape of train_data_hot_encoded is ", train_data_hot_encoded.shape)
print("Shape of test_data_hot_encoded is ", test_data_hot_encoded.shape)


Problem solved!

Our categorial training data is now one-hot encoded in `train_data_hot_encoded`<br>
Our categorial test data is now one-hot encoded in `test_data_hot_encoded`

Both one-hot encoded data matrices have 275 feature columns.

# Scaling the data and preparing training matrices

When we just used numerical features for predicting house sale prices, we rescaled all the numerical features to range [0,1] in order to make it easier for the MLP to learn appropriate weights.

Here - where we use just categorial one-hot encoded features - a scaling of the input data is not needed! All values are 0 or 1. However, we need to prepare NumPy matrices and scale the output (sale price).

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. define how NumPy shall print matrices
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True) # do not use scientific "e"-notation

# 2. convert Pandas DataFrame to NumPy matrices
#    since Keras will expect NumPy matrices
cat_train_input_matrix = train_data_hot_encoded.values
train_output_matrix    = gt_saleprice.values
train_output_matrix    = train_output_matrix.reshape(-1,1)
cat_test_input_matrix  = test_data_hot_encoded.values
print("Type of cat_train_input_matrix is", type(cat_train_input_matrix))
print("Shape of cat_train_input_matrix is", cat_train_input_matrix.shape)
print("Shape of train_output_matrix is", train_output_matrix.shape)
print("Shape of cat_test_input_matrix is", cat_test_input_matrix.shape)
print("train_data_hot_encoded:\n", train_data_hot_encoded)
print("cat_train_input_matrix:\n", cat_train_input_matrix)

# 3. create a MinMaxScaler for the train_output_matrix,
#    which is essentially a column with the final SalePrice
scaler_saleprice = MinMaxScaler(feature_range=(0, 1))
normalized_train_output_matrix = scaler_saleprice.fit_transform(train_output_matrix)
print("\ntrain_output_matrix:\n", train_output_matrix)
print("\nnormalized_train_output_matrix:\n", normalized_train_output_matrix)

# Building the MLP model and training it

Now we can use the training data to train a MLP.

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.layers.core import Dense, Dropout
import pandas as pd
import numpy as np

model = Sequential()
model.add(Dense(160, activation="linear"))
model.add(Dense(80, activation="linear"))
model.add(Dense(1, activation="linear"))

model.compile(loss='mean_squared_error', optimizer='adam')

X = cat_train_input_matrix
Y = normalized_train_output_matrix
print("Input X has shape", X.shape)
print("Desired output Y has shape", Y.shape)
print("Y:\n", Y)
model.fit(X,Y, validation_split=0.10, epochs=100)

# Testing the trained MLP

Normally we test a model on a separate test data set. However, in this case, there is no test dataset, since the test data only contains the input features, but not the final sale prices of the houses.

For this, let us see how good the trained model can predict the sale prices of the houses from the training dataset ONLY using the categorial features.

In [ ]:
preds_train_houses = model.predict(cat_train_input_matrix)
print("preds_train_houses:\n", preds_train_houses)
preds_train_houses_dollar = scaler_saleprice.inverse_transform(preds_train_houses)
print("preds_train_houses_dollar:\n", preds_train_houses_dollar)
print("Shape of preds_train_houses is", preds_train_houses.shape)
print("Shape of preds_train_houses_dollar is", preds_train_houses_dollar.shape)
plt.figure( figsize=(10,10) )
plt.plot(train_output_matrix, preds_train_houses_dollar, 'ro')
plt.xlabel('Real house price', fontsize = 10)
plt.ylabel('Predicted house price', fontsize = 10)
plt.grid(True)
plt.xlim(0,900000)
plt.ylim(0,900000)
plt.show()

Let us now compute the average prediction error made in Dollars:

In [ ]:
print("preds_train_houses_dollar:\n", preds_train_houses_dollar)
print("train_output_matrix:\n", train_output_matrix)
absdiff = np.abs(preds_train_houses_dollar - train_output_matrix)
print(absdiff)
average_error_in_dollar = np.mean(absdiff)
print("The final trained model made an average error of $%.2f" % average_error_in_dollar,
      "when predicting the house prices on the training data when ONLY using"
      "categorial features.")

# Predicting house prices for the Kaggle competition

Now we will use the trained MLP to predict the prices of the houses for which only the features are given, but not the sale price!

This file can be uploaded at Kaggle to take part at the Kaggle competition!

In [ ]:
# PREDICT house prices for all the test houses!
preds_test_houses = model.predict(cat_test_input_matrix)
preds_test_houses_dollar = scaler_saleprice.inverse_transform(preds_test_houses)


# generate a Pandas dataframe
# from the NumPy prediction_matrix
preds_test_houses_dollar = preds_test_houses_dollar.reshape(-1)
print("test_house_ids has shape", test_house_ids.shape)
print("preds_test_houses_dollar has shape", preds_test_houses_dollar.shape)
predition_dataframe = pd.DataFrame({'Id'       :test_house_ids,
                                    'SalePrice':preds_test_houses_dollar}
                                  )

# convert column "Id" to int64 dtype
predition_dataframe = predition_dataframe.astype({"Id": int})
print(predition_dataframe)

# now save the Pandas dataframe to a .csv file
PREDICTION_FILENAME = "my_predicted_house_prices.csv"
predition_dataframe.to_csv(PREDICTION_FILENAME, sep=',', index=False)